In [165]:
import os
import requests
from bs4 import BeautifulSoup

Just download all the (ranked) listings for a given country.

It looks like this returns a maximum of 128 pages (each 20 PV systems), for a total of 128 x 20 = 2560 systems.  But that's OK, because the list is ranked, so we should get the most important PV systems.

In [190]:
COUNTRY_CODES = {'UK': 243}
COUNTRY = 'UK'
BASE_URL = 'https://pvoutput.org/map.jsp'
DEST_PATH = '/home/jack/data/pvoutput.org/raw'

In [192]:
country_code = COUNTRY_CODES[COUNTRY]

def get_html(page):
    query_string = 'p={p}&country={country_code}'.format(p=page, country_code=country_code)
    url = BASE_URL + '?' + query_string
    return requests.get(url)


def save_response_to_disk(response, page):
    filename = os.path.join(DEST_PATH, 'pvoutput_{}_page_{:03d}.html'.format(COUNTRY, page))
    with open(filename, mode='w') as fh:
        fh.write(response.text)

def page_has_next_link(response):
    soup = BeautifulSoup(response.text, 'html.parser')
    return bool(soup.find_all('a', text='Next'))


page = 6
while True:
    print('\r', page, end='', flush=True)
    response = get_html(page=page)
    save_response_to_disk(response, page=page)
    if not page_has_next_link(response):
        break
    page += 1

 127